In [62]:
import os

import QuantLib as ql
import numpy as np
import pandas as pd

In [29]:
def hestonPricing(callput,calculation_date,maturity_date,spot_price,strike_price,volatility,dividend_rate,risk_free_rate,
                  kappa,theta,sigma,rho):
    if callput == 'call':
        option_type = ql.Option.Call
    else: 
        option_type = ql.Option.Put
    
    #Day count settings in QL
    day_count = ql.Actual365Fixed()
    ql.Settings.instance().evaluationDate = calculation_date
    
    # construct the option payoff
    payoff = ql.PlainVanillaPayoff(option_type, strike_price)
    exercise = ql.EuropeanExercise(maturity_date)
    european_option = ql.VanillaOption(payoff, exercise)
    
    # set the remaining Heston parameters
    v0 = volatility*volatility # spot variance
    spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))
    
    # construct the Heston process
    rf_rate = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date,
                                                         risk_free_rate, day_count))

    dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date,
                                                                dividend_rate, day_count))

    heston_process = ql.HestonProcess(rf_rate, dividend_yield,
                                      spot_handle, v0, kappa,
                                      theta, sigma, rho)
    
    # run the pricing engine
    engine = ql.AnalyticHestonEngine(ql.HestonModel(heston_process),10**(-4), 10**(4))
    european_option.setPricingEngine(engine)
    
    return european_option.NPV()
        
    

In [57]:
def createGridSpace(arrays, out=None):
    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = int(n / arrays[0].size) 
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        createGridSpace(arrays[1:], out=out[0:m, 1:])
        for j in range(1, arrays[0].size):
            out[j*m:(j+1)*m, 1:] = out[0:m, 1:]
    return out

### The heston price for given inputs

In [58]:
callput = 'call' #call or put
calculation_date = ql.Date(26, 6, 2020)
maturity_date = ql.Date(30, 6, 2020)
spot_price = 969.74
strike_price = 1000
volatility = 0.2
dividend_rate = 0.0
risk_free_rate = 0.0
kappa = 0.1
theta = 0.01
sigma = 0.1
rho = -0.75

In [59]:
hestonPricing(callput,calculation_date,maturity_date,spot_price,strike_price,volatility,dividend_rate,risk_free_rate,
                  kappa,theta,sigma,rho)

0.6091512720622063

### Create the Heston GRID

In [60]:
kappa = np.linspace(0.1,1,10)
theta = np.linspace(0.1,1,10)
sigma = np.linspace(0.1,1,10)
rho = np.linspace(0.1,1,10)
volatility = np.linspace(0.1,1,10)
grid = createGridSpace([kappa,theta,sigma,rho,volatility])

In [67]:
len(grid)

100000

### The xeo (sp100 eu) data

In [64]:
xeo = pd.read_csv('xeo option data 2000-2020.csv')

In [70]:
xeoSpot = pd.read_csv('xeoSpotPrice.csv') #=GOOGLEFINANCE("XEO", "price", DATE(2001,7,20), DATE(2020,12,31), "DAILY")

In [72]:
xeo.head(5)

,secid,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,impl_volatility,optionid,forward_price,index_flag,issuer,exercise_style
0,112878,20010723,20010818,NaN,C,550000,64.5,66.5,0,0.212728,20418435,NaN,1,CBOE S&P 100 INDEX,E
1,112878,20010723,20010818,NaN,C,560000,55.3,57.3,0,0.238091,20418436,NaN,1,CBOE S&P 100 INDEX,E
2,112878,20010723,20010818,NaN,C,570000,46.4,48.4,0,0.243815,20418437,NaN,1,CBOE S&P 100 INDEX,E
3,112878,20010723,20010818,NaN,C,580000,37.8,39.8,0,0.240366,20418438,NaN,1,CBOE S&P 100 INDEX,E
4,112878,20010723,20010818,NaN,C,590000,29.9,31.9,0,0.237942,20418439,NaN,1,CBOE S&P 100 INDEX,E


In [71]:
xeoSpot.head(5)

,Date,Close
0,30/04/2004 15:16:00,540.88
1,03/05/2004 15:16:00,545.85
2,04/05/2004 15:16:00,547.15
3,05/05/2004 15:16:00,547.19
4,06/05/2004 15:16:00,543.90
